In [ ]:
%pylab inline
from __future__ import division, print_function
from statsmodels.kernel_methods import kde, kde_methods, kernels, bootstrap
from statsmodels.kernel_methods import _fast_linbin as fast_linbin
from scipy import stats

In [ ]:
d1 = stats.norm(1)
r = d1.rvs(100)
print("r.shape = ", r.shape)
k = kde.KDE(r, method=kde_methods.Reflection1D(), bandwidth=kde.bandwidths.scotts)
est = k.fit()

In [ ]:
mesh, bins = fast_linbin.fast_linbin(r, [-3, 5], 110)
norm_bins = bins / (sum(bins) * mesh.start_volume)
f = figure()
bar(asarray(mesh), norm_bins, mesh.start_interval[0], align='center', edgecolor='none')
xlim(xmin=-3, xmax=5)

In [ ]:
mesh.integrate(bins)

In [ ]:
pts = r_[-3:5:1024j]
pts = c_[pts[:512], pts[512:]]
pts.shape

### Testing per-dim functions

In [ ]:
pdf1 = np.empty(pts.shape, dtype=pts.dtype)
est.pdf(pts[:,0], out=pdf1[:,0])
est.pdf(pts[:,1], out=pdf1[:,1])
pdf1.shape

In [ ]:
f = figure()
plot(pts[:,1], pdf1[:,0])
plot(pts[:,0], pdf1[:,1])

In [ ]:
xs, pdf = est.grid(2**10)

In [ ]:
f = figure()
bar(np.asarray(mesh), norm_bins, mesh.start_interval[0], align='center', edgecolor='none')
plot(np.asarray(xs), pdf, color='red', label='KDE')
xlim(-3, 5)
legend(loc='best')

In [ ]:
est.kernel = kernels.Epanechnikov()
xs, pdf = est.grid(2**10)
f = figure()
bar(mesh.full(), norm_bins, mesh.start_interval[0], align='center', edgecolor='none')
plot(xs.full(), pdf, color='red', label='KDE')
xlim(-3, 5)
legend(loc='best')

In [ ]:
est.kernel = kde.kernels.normal1d()
from_binned = est.from_binned(mesh, bins, normed=True)

In [ ]:
from_binned.max()

In [ ]:
f = figure()
bar(mesh.full(), norm_bins, mesh.start_interval[0], align='center', edgecolor='none')
plot(mesh.full(), from_binned, color='red', label='KDE')
xlim(-3, 5)
legend(loc='best')

#### Bootstrapping in 1D

In [ ]:
small_mesh, small_values = est.grid(N=1024)
_, values = bootstrap.bootstrap_grid(est, 3000, CIs=(0, 0.95, 0.99), fct_args=dict(N=1024))

In [ ]:
values.shape

In [ ]:
f = figure()
#bar(mesh.full(), norm_bins, mesh.start_interval[0], align='center', edgecolor='none')
plot(small_mesh.full(), small_values, color='red', label='KDE')
plot(small_mesh.full(), values[:,0,0], color='green', label='KDE median')
plot(small_mesh.full(), values[:,1,0], color='red', linestyle='--', label='CI (95%)')
plot(small_mesh.full(), values[:,1,1], color='red', linestyle='--')
fill_between(small_mesh.full(), values[:,1,0], values[:,1,1], color='red', alpha=0.2)
plot(small_mesh.full(), values[:,2,0], color='gray', linestyle='--', label='CI (99%)')
plot(small_mesh.full(), values[:,2,1], color='gray', linestyle='--')
xlim(-3, 5)
legend(loc='best')

In [ ]:
_, values = bootstrap.bootstrap_grid(est, 3000, CIs=(0.5, 0.95, 0.99), fct_args=dict(N=1024))

### 2D case

In [ ]:
N2=1e4
r2 = c_[d1.rvs(N2), 0.5*d1.rvs(N2)]

In [ ]:
mesh2, bins2 = fast_linbin.fast_linbin_nd(r2, [[-3, 5.5], [-3, 5]], [128,256])
bins2 /= bins2.sum()
est.bandwidth = 0.4
mesh2.integrate(bins2)

In [ ]:
f = figure()
m = mesh2.full('C')
pcolormesh(m[0], m[1], bins2, cmap=cm.jet, shading='flat')
axis('equal')
xlim(-3, 5)
ylim(-3, 5)
colorbar()


In [ ]:
from_bins_x = est.from_binned(mesh2, bins2, dim=0)
s2x = mesh2.integrate(from_bins_x)
s2x / mesh2.start_interval[1]

In [ ]:
f = figure()
m = mesh2.full('C')
pcolormesh(m[0], m[1], from_bins_x, cmap=cm.jet, shading='flat')
axis('equal')
xlim(-3, 5)
ylim(-3, 5)
colorbar()

In [ ]:
from_bins_y = est.from_binned(mesh2, from_bins_x, dim=1)
s2y = mesh2.integrate(from_bins_y)
s2y

In [ ]:
f = figure()
m = mesh2.full('C')
pcolormesh(m[0], m[1], from_bins_y, cmap=cm.jet, shading='flat')
axis('equal')
xlim(-3, 5)
ylim(-3, 5)
colorbar()

### And in 3D ...

In [ ]:
N3=1e4
r3 = c_[d1.rvs(N3), 0.5*d1.rvs(N3), 0.7*d1.rvs(N3)]

In [ ]:
M = 256
mesh3, bins3 = fast_linbin.fast_linbin_nd(r3, [[-3, 5.5], [-3, 5], [-3, 5]], M)
bins3 /= bins3.sum()
est.bandwidth = 0.4
mesh3.integrate(bins3)

In [ ]:
from_bins3_x = est.from_binned(mesh3, bins3, dim=0)
s3x = mesh3.integrate(from_bins3_x)
s3x / prod(mesh3.start_interval[1:])

In [ ]:
from_bins3_y = est.from_binned(mesh3, from_bins3_x, dim=1)
s3y = mesh3.integrate(from_bins3_y)
s3y / prod(mesh3.start_interval[2])

In [ ]:
from_bins3_z = est.from_binned(mesh3, from_bins3_y, dim=2)
s3z = mesh3.integrate(from_bins3_z)
s3z

In [ ]:
f = figure()
m = mesh3.full('C')
pcolormesh(m[0,:,:,M/2], m[1,:,:,M/2], from_bins3_z[:,:,M/2], cmap=cm.jet, shading='flat')
axis('equal')
xlim(-3, 5)
ylim(-3, 5)
xlabel('X')
ylabel('Y')
colorbar()

In [ ]:
f = figure()
m = mesh3.full('C')
pcolormesh(m[0,:,M/2,:], m[2,:,M/2,:], from_bins3_z[:,M/2,:], cmap=cm.jet, shading='flat')
axis('equal')
xlim(-3, 5)
ylim(-3, 5)
xlabel('X')
ylabel('Z')
colorbar()

In [ ]:
f = figure()
m = mesh3.full('C')
pcolormesh(m[1,M/2,:,:], m[2,M/2,:,:], from_bins3_z[M/2,:,:], cmap=cm.jet, shading='flat')
axis('equal')
xlim(-3, 5)
ylim(-3, 5)
xlabel('Y')
ylabel('Z')
colorbar()